In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR,LinearSVR
from sklearn.neighbors import KNeighborsClassifier
#from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
#from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler

In [19]:
df=pd.read_csv("file:///C:/Users/00824732/Desktop/supermarket_sales%20-%20Sheet1.csv")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [21]:
def pre_process(df):
    df=df.copy()
    y=df['Rating']
    x=df.drop('Rating',axis=1)
    return x,y

In [22]:
x,y=pre_process(df)

In [23]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=3)

In [1]:
## Constructing a pipeline

In [97]:
df.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

In [31]:
{column:len(x_train[column].unique()) for column in x_train.select_dtypes('object').columns}

{'Invoice ID': 700,
 'Branch': 3,
 'City': 3,
 'Customer type': 2,
 'Gender': 2,
 'Product line': 6,
 'Date': 89,
 'Time': 418,
 'Payment': 3}

In [ ]:
## Categorize our featuers

In [99]:
binary_featuers=[
    'Customer type',
    'Gender'
]
date_featuers=[
    'Date'
]
time_featuers=[
    'Time'
]
nominal_featuers=   [
    'Branch',
    'City',
    'Product line',
    'Payment'
]

In [25]:
pd.get_dummies(x_train['Branch'])

,A,B,C
926,0,1,0
411,0,1,0
224,1,0,0
14,1,0,0
170,1,0,0
...,...,...,...
952,0,1,0
643,0,0,1
249,0,1,0
664,0,0,1


In [92]:
## create custom transformers for date and time featuers

In [109]:
class DateEncoder:
    def fit(self,x,y):
        return self
    def transform(self,x):
        for column in x.columns:
            x[column]=pd.to_datetime(x[column])
            x[column+'_year']=x[column].apply(lambda x:x.year)
            x[column+'_month']=x[column].apply(lambda x:x.month)
            x[column+'_day']=x[column].apply(lambda x:x.day)
            x=x.drop(column,axis=1)
        return x

In [101]:
pd.to_datetime(x_train['Time']).apply(lambda x:x.day)
pd.to_datetime(x_train['Time']).apply(lambda x:x.month)

926    2
411    2
224    2
14     2
170    2
      ..
952    2
643    2
249    2
664    2
874    2
Name: Time, Length: 700, dtype: int64

In [102]:
DateEncoder().transform(x_train[['Time']])

,Time_year,Time_month,Time_day
926,2023,2,17
411,2023,2,17
224,2023,2,17
14,2023,2,17
170,2023,2,17
...,...,...,...
952,2023,2,17
643,2023,2,17
249,2023,2,17
664,2023,2,17


In [110]:
class TimeEncoder:
    def fit(self,x,y):
        return self
    def transform(self,x):
        for column in x.columns:
            x[column]=pd.to_datetime(x[column])
            x[column+'_hour ']=x[column].apply(lambda x:x.hour)
            x[column+'_minute']=x[column].apply(lambda x:x.minute)
            x=x.drop(column,axis=1)
        return x

In [111]:
## Construct transformer pipelines for each feature type
binary_transformer=Pipeline(steps=[('ordinal',OrdinalEncoder())])
date_transformer=Pipeline(steps=[('date',DateEncoder())])
time_transformer=Pipeline(steps=[('time',TimeEncoder())])
nominal_transformer=Pipeline(steps=[('onehot',OneHotEncoder())])

In [112]:
## Combine transfomers with Column Transformer
preprocessor=ColumnTransformer(transformers=[('binary',binary_transformer,binary_featuers),(
    'date',date_transformer,date_featuers),
    ('time',time_transformer,time_featuers),
    ('nominal',nominal_transformer,nominal_featuers)
])

In [157]:
## Training

In [114]:
models={
    "LinearRegression": LinearRegression(),
    "SVM(Linear Kernel)":LinearSVR()
}
scaler=StandardScaler()

In [117]:
for name,model in models.items():
    pipeline=Pipeline(steps=[('preprocessor',preprocessor),('scaler',scaler),('regressor',model)])
# ## Fit pipeline
pipeline.fit(x_train,y_train)   
print(name+"trained")

SVM(Linear Kernel)trained


In [124]:
## Results
for name,model in models.items():
    pipeline=Pipeline(steps=[
        ('preprocessor',preprocessor),
        ('scaler',scaler),
        ('regressor',model)
    ])
print(name+"R 2 square:{:.5f}".format(pipeline.score(x_test,y_test)))

SVM(Linear Kernel)R 2 square:-0.08459


## Second Project

In [37]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [128]:
df=pd.read_csv("file:///C:/Users/00824732/Desktop/New%20York%20City%20Restaurant%20Inspection.csv")

In [151]:
pd.set_option('display.max_columns',None)
df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,41089834,MURRAY HILL DINER,Manhattan,222,LEXINGTON AVENUE,10016.0,2126866667,NaN,NaN,P,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,40.745891,-73.980104,105.0,2.0,7200.0,1018492.0,1.008890e+09,MN20
1,41089834,MURRAY HILL DINER,Manhattan,222,LEXINGTON AVENUE,10016.0,2126866667,NaN,NaN,P,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,40.745891,-73.980104,105.0,2.0,7200.0,1018492.0,1.008890e+09,MN20
2,41089834,MURRAY HILL DINER,Manhattan,222,LEXINGTON AVENUE,10016.0,2126866667,NaN,NaN,P,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,40.745891,-73.980104,105.0,2.0,7200.0,1018492.0,1.008890e+09,MN20
3,41089834,MURRAY HILL DINER,Manhattan,222,LEXINGTON AVENUE,10016.0,2126866667,NaN,NaN,P,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,40.745891,-73.980104,105.0,2.0,7200.0,1018492.0,1.008890e+09,MN20
4,40596549,POLLOS A LA BARASA MARIO,Queens,40-19,BROADWAY,11103.0,7189322424,NaN,NaN,D,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,40.758872,-73.918772,401.0,22.0,15500.0,4011093.0,4.006780e+09,QN70


In [130]:
{column:len(x[column].unique()) for column in x.columns}

{'Invoice ID': 1000,
 'Branch': 3,
 'City': 3,
 'Customer type': 2,
 'Gender': 2,
 'Product line': 6,
 'Unit price': 943,
 'Quantity': 10,
 'Tax 5%': 990,
 'Total': 990,
 'Date': 89,
 'Time': 506,
 'Payment': 3,
 'cogs': 990,
 'gross margin percentage': 1,
 'gross income': 990}

In [109]:
df=df.iloc[0:1000]

In [135]:
## Preprocessing
def preprocess_inputs(df):
    df=df.copy()
    ## Drop CAMIS and DBA Columns
    df['zipcode']=df['ZIPCODE'].astype(str)
    ##Dropping single value column
    df=df.drop('RECORD DATE',axis=1)
    ## DROPPING HIGH CARDINALITY COLUMN
    high_cardinal=[
        'BUILDING',
        'STREET',
        'ZIPCODE',
     'PHONE'
    ]
    df=df.drop(high_cardinal,axis=1)
    ## Extract datefeatuers
    df['INSPECTION DATE']=pd.to_datetime(df['INSPECTION DATE'])
    df['INSPECTION YEAR']=pd.to_datetime(df['INSPECTION DATE'].apply(lambda x:x.year))
    df['INSPECTION MONTH']=pd.to_datetime(df['INSPECTION DATE'].apply(lambda x:x.month))
    df.drop('INSPECTION DATE',axis=1)
    df['CRITICAL FLAG']=df['CRITICAL FLAG'].astype(object)
    missing_values=df.columns[df.isna().mean()>=0.25]
    df=df.drop(missing_values,axis=1)
   ## Droprows with missing target values
    missing_targets=df[df['CRITICAL FLAG']=='Not Applicable'].index
    df=df.drop(missing_targets,axis=0).reset_index(drop=True)
    ## One hot encoding
    for column in df.select_dtypes('object').drop('CRITICAL FLAG',axis=1):
        dummies=pd.get_dummies(df[column],prefix=column)
        df=pd.concat([df,dummies],axis=1)
        df=df.drop(column,axis=1)
    x=df.drop("CRITICAL FLAG",axis=1)
    y=df["CRITICAL FLAG"]
    x_train,x_test,y_train,y_test=train_test_split(x,y)
    ## scale
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),index=x_train.index,columns=x_train.columns)
    x_test=pd.DataFrame(scaler.transform(x_test),index=x_test.index,columns=x_test.columns) 
    return x_train,x_test,y_train,y_test

In [ ]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)

In [131]:
model=LogisticRegression()

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.coef_

In [ ]:
acc=model.score(x_test,y_test)
print("Test accuracy :{.2f}%".format(acc*100))

In [ ]:
y_pred=model.predict(x_test)    
cm=confusion_matrix(y_test,y_pred,labels=['Not Critical','Critical'])
clr=classification_report(y_test,y_pred,labels=['Not Critical','Critical'])
sns.heatmap(cm,annot=True)
plt.show()
print("classification report",clr)

In [ ]:
## Feature importance
## use weights to model to see the most importance
most_importance=np.argmax(model.coef_)
x_train.columns[np.argmax(model.coef_)]

In [ ]:
df.query('VIOLATION CODE'=='10F')['VIOLATION DESCRIPTION']

In [ ]:
## check the description associated
code_description=df.query('VIOLATION CODE'=='10F')['VIOLATION DESCRIPTION']
print("code"+code_description)